In [0]:
import tensorflow as tf
import  keras
from  keras  import  Sequential
from keras.layers import Dense
from keras.utils import to_categorical as onehot
import h5py
import sys
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split as split
from google.colab import drive

In [0]:
drive.mount('/content/drive')
sys.path.append('drive/My Drive/Colab Notebooks/')

In [0]:
test_data_path = 'drive/My Drive/Colab Notebooks/test_guitar_vs_nonguitar.h5'
train_data_path = 'drive/My Drive/Colab Notebooks/train_guitar_vs_nonguitar.h5'
data_train = h5py.File(train_data_path, 'r')
data_test = h5py.File(test_data_path, 'r')
X = data_train['train_set_x'].value.flatten().reshape(300, 64*64*3)
y = data_train['train_set_y'].value
X = np.concatenate((X, data_test['test_set_x'].value.flatten().reshape(200, 64*64*3)), axis = 0)
y = np.concatenate((y, data_test['test_set_y'].value), axis = 0)
y  = onehot(y, 2)

In [0]:
X_train, X_test, y_train, y_test = split(X, y, test_size = 0.2, random_state = 1511849)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(400, 12288)
(100, 12288)
(400, 2)
(100, 2)


In [0]:
n_input = 12288
n_classes = 2
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [0]:
n_hidden_1 = 1200 
n_hidden_2 = 500 
n_hidden_3 = 100 

weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'w_out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b_out': tf.Variable(tf.random_normal([n_classes]))
}

In [0]:
def multilayer_perceptron(x):
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    
    output_layer = tf.matmul(layer_3, weights['w_out']) + biases['b_out']
    
    return output_layer

In [0]:
logits = multilayer_perceptron(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
learning_rate = 0.005
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
init = tf.global_variables_initializer()

In [19]:
n_epochs = 300
batch_size = 100

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        avg_cost = 0.0
        total_batch = 400
        for i in range(total_batch):
            _, cost = sess.run([train_op, loss_op], 
                            feed_dict={X: X_train, Y: y_train})
            avg_cost += cost / total_batch
    print("Finished training!")

    pred = tf.nn.softmax(logits)  
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: X_test, Y: y_test}))

Finished training!
Accuracy: 0.74
